In [261]:
import numpy as np
import pandas as pd
from requests import request
import os
from math import ceil

from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

url = 'https://sandbox.iexapis.com/stable/stock/twtr/chart/1y/20190109?chartByDay=true&token=Tsk_d19a63cb677f46e3a663e48d3bcf38ed'

startdate = '20200515' # 10/14/2020 = wednesday

stock = 'ISBC'

In [262]:
# calculate the number of months to look back
#untested
months_lookback = math.ceil((datetime.now() - pd.to_datetime(startdate)).days / 31)

#calculate the number of years to lookback
#tested and works
_lookback = math.ceil((datetime.now() - pd.to_datetime(startdate)).days / 31)

In [265]:

#which data source to load from
env = 'cloud'

if env=='sandbox':
    source = 'sandbox'
    tkn=os.getenv('iex_secret_sandbox')
elif env=='cloud':
    source='cloud'
    tkn=os.getenv('iex_secret')

url = f'https://{source}.iexapis.com/stable/stock/{stock}/chart/{months_lookback}m?chartCloseOnly=true&chartByDay=true&token={tkn}'

In [266]:
# load data from iexcloud.io
#response = requests.request(url=url, method='get')

# convert iexcloud.io to pandas dataframe
#df = pd.read_json(response.content)

# save df to a pickle file
#df.to_pickle(f'{stock}_prices_{startdate}.pickle')

# load the data from the saved pickle
df = pd.read_pickle(f'{stock}_prices_{startdate}.pickle')


In [267]:
#set the date column to the index
df = df.set_index(pd.to_datetime(df.date), format='%Y%d%m')

# delete the date value column
df = df.drop(columns=['date'])

In [268]:
#filter for the correct period
df = df.loc[pd.to_datetime(startdate) + pd.offsets.Week(-1):]

In [269]:
# check data
df.loc['2020-05-12':][:5]

,close,volume,changeOverTime,marketChangeOverTime,label,change,changePercent
date,,,,,,,
2020-05-12,8.070,1526327,-0.305508,-0.305508,"May 12, 20",-0.450,-0.0528
2020-05-13,7.600,1561420,-0.345955,-0.345955,"May 13, 20",-0.470,-0.0582
2020-05-14,7.605,2269355,-0.345525,-0.345525,"May 14, 20",0.005,0.0007
2020-05-15,7.620,1961884,-0.344234,-0.344234,"May 15, 20",0.015,0.0020
2020-05-18,8.390,1522307,-0.277969,-0.277969,"May 18, 20",0.770,0.1010


In [272]:
startdate

'20200515'

In [291]:
pd.to_datetime(startdate, infer_datetime_format=True)

Timestamp('2020-05-15 00:00:00')

In [316]:
frequency = f'W-{pd.to_datetime(startdate, infer_datetime_format=True).day_name()[:3]}'

In [337]:
# we are looking for a weekly offset
offset = pd.offsets.Week(-1)

# resample for weekly series


res = df.close.loc[pd.to_datetime(startdate)+offset:].resample(frequency,loffset=offset, closed='left').first()

In [338]:
res = pd.DataFrame(res)

In [342]:
res['chg'] = np.log(res.close/res.close.shift(1))

In [343]:
res

,close,chg
date,,
2020-05-08,8.850,NaN
2020-05-15,7.620,-0.149641
2020-05-22,8.190,0.072138
2020-05-29,8.680,0.058108
2020-06-05,9.700,0.111104
2020-06-12,8.570,-0.123858
2020-06-19,8.205,-0.043524
2020-06-26,7.880,-0.040416
2020-07-03,8.190,0.038586


In [302]:
df.loc['2020-5-10':][:10]

,close,volume,changeOverTime,marketChangeOverTime,label,change,changePercent
date,,,,,,,
2020-05-11,8.520,2389206,-0.266781,-0.266781,"May 11, 20",-0.330,-0.0373
2020-05-12,8.070,1526327,-0.305508,-0.305508,"May 12, 20",-0.450,-0.0528
2020-05-13,7.600,1561420,-0.345955,-0.345955,"May 13, 20",-0.470,-0.0582
2020-05-14,7.605,2269355,-0.345525,-0.345525,"May 14, 20",0.005,0.0007
2020-05-15,7.620,1961884,-0.344234,-0.344234,"May 15, 20",0.015,0.0020
2020-05-18,8.390,1522307,-0.277969,-0.277969,"May 18, 20",0.770,0.1010
2020-05-19,7.970,1014856,-0.314114,-0.314114,"May 19, 20",-0.420,-0.0501
2020-05-20,8.360,1512344,-0.280551,-0.280551,"May 20, 20",0.390,0.0489
2020-05-21,8.290,841747,-0.286575,-0.286575,"May 21, 20",-0.070,-0.0084


In [225]:
res.head()

date
2020-10-14    7.86
2020-10-21    7.87
2020-10-28    8.19
2020-11-04    8.47
2020-11-11    9.30
Freq: W-WED, Name: close, dtype: float64

In [163]:
res[]

Index(['Friday', 'Friday', 'Friday', 'Friday', 'Friday', 'Friday', 'Friday',
       'Friday', 'Friday', 'Friday',
       ...
       'Friday', 'Friday', 'Friday', 'Friday', 'Friday', 'Friday', 'Friday',
       'Friday', 'Friday', 'Friday'],
      dtype='object', name='date', length=261)